In [1]:
# Run once if needed
# !{sys.executable} -m pip install chromadb sentence-transformers transformers accelerate


In [2]:
import os
import textwrap

import chromadb
from chromadb.utils import embedding_functions

from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import pipeline

import torch
import numpy as np

print("Python:", os.popen("which python").read().strip())
print("CUDA available:", torch.cuda.is_available())
device = "cuda" if torch.cuda.is_available() else "cpu"
device


Python: /home/christopher_bonillajulien_22/miniconda3/envs/torch_env/bin/python
CUDA available: True


'cuda'

In [3]:
#Load Chroma & Final Triplet Retriever
# 1. Load final triplet-trained retriever
RETRIEVER_MODEL_PATH = "models/pg16-minilm-triplet"

retriever = SentenceTransformer(RETRIEVER_MODEL_PATH)
retriever = SentenceTransformer("models/pg16-minilm-triplet")
retriever.to("cpu")
retriever = retriever.eval()
print("Loaded retriever:", RETRIEVER_MODEL_PATH)

# 2. Connect to Chroma (must match Notebook 0/2/3)
CHROMA_DIR = "chroma_pg16_minilm"
COLLECTION_NAME = "pg16_minilm"

client = chromadb.PersistentClient(path=CHROMA_DIR)
collection = client.get_collection(name=COLLECTION_NAME)

print("Connected to Chroma collection:", COLLECTION_NAME)
#Chroma already has embeddings from Stage 2. We’ll use the triplet model to embed the queries, and Chroma’s stored vectors for passages.

Loaded retriever: models/pg16-minilm-triplet
Connected to Chroma collection: pg16_minilm


In [4]:
#Load Reranker (CrossEncoder)
from sentence_transformers import CrossEncoder

# CPU RERANKER (safe for all GPUs)
RERANKER_MODEL_NAME = "BAAI/bge-reranker-v2-m3"

reranker = CrossEncoder(RERANKER_MODEL_NAME, device="cpu")  
print("Loaded reranker on CPU:", RERANKER_MODEL_NAME)


Loaded reranker on CPU: BAAI/bge-reranker-v2-m3


In [5]:
#Load Generator LLM
from transformers import pipeline
import torch

GENERATOR_MODEL_NAME = "Qwen/Qwen2-0.5B-Instruct"

generator = pipeline(
    "text-generation",
    model=GENERATOR_MODEL_NAME,
    device=-1,         # CPU ONLY
    torch_dtype=torch.float32,
)

print("Loaded generator on CPU:", GENERATOR_MODEL_NAME)


`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cpu


Loaded generator on CPU: Qwen/Qwen2-0.5B-Instruct


In [6]:
#Retriever Helper (Triplet Model + Chroma)
def retrieve_with_triplet(
    question: str,
    k_retriever: int = 20,
    include: list[str] = ["documents", "metadatas", "distances"]
):
    """Use triplet model to embed query, then search Chroma by embedding."""
    # 1. Encode query
    q_text = f"query: {question}"
    q_emb = retriever.encode(q_text, convert_to_numpy=True)
    
    # 2. Query Chroma by embedding
    result = collection.query(
        query_embeddings=[q_emb.tolist()],
        n_results=k_retriever,
        include=include
    )
    
    return result


In [7]:
#Reranking Helper
def rerank_results(question: str, retrieval_result, k_rerank: int = 5):
    docs = retrieval_result["documents"][0]
    metas = retrieval_result["metadatas"][0]
    
    # Build (query, doc) pairs for reranker
    pairs = [(question, d) for d in docs]
    
    # Predict relevance scores
    scores = reranker.predict(pairs)  # shape: [num_docs]
    
    # Sort by score descending
    idx_sorted = np.argsort(scores)[::-1]
    
    reranked = []
    for i in idx_sorted[:k_rerank]:
        reranked.append({
            "doc": docs[i],
            "meta": metas[i],
            "score": float(scores[i])
        })
    return reranked


In [8]:
#Build Prompt from Context
def build_prompt(question: str, contexts: list[dict]):
    """
    contexts: list of dicts with keys: doc, meta, score
    """
    context_blocks = []
    for i, c in enumerate(contexts):
        page = c["meta"].get("page", "?")
        header = f"[Doc {i+1} — page {page}, score={c['score']:.3f}]"
        body = textwrap.fill(c["doc"], width=100)
        context_blocks.append(header + "\n" + body)
    
    context_text = "\n\n".join(context_blocks)
    
    prompt = f"""
You are a helpful assistant answering questions about PostgreSQL 16 documentation.

Use the CONTEXT below to answer the QUESTION. 
If the answer is not in the context, clearly say you don't know, 
and do NOT hallucinate.

CONTEXT:
{context_text}

QUESTION:
{question}

Answer in a clear, concise explanation. 
If relevant, refer to [Doc #] when citing specific info.
"""
    return prompt.strip()


In [9]:
#Generator Wrapper
def generate_answer(prompt: str, max_new_tokens: int = 256):
    outputs = generator(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.3,
        top_p=0.9
    )
    # Qwen returns list of dicts with "generated_text"
    text = outputs[0]["generated_text"]
    # Some models echo the prompt; trim if necessary:
    if prompt in text:
        text = text[len(prompt):].strip()
    return text.strip()


In [10]:
#Full RAG Pipeline: answer_question
def answer_question(
    question: str,
    k_retriever: int = 20,
    k_rerank: int = 5,
    k_context: int = 4
):
    # 1. Retrieve
    retrieval_result = retrieve_with_triplet(question, k_retriever=k_retriever)
    
    # 2. Rerank
    reranked = rerank_results(question, retrieval_result, k_rerank=k_rerank)
    
    # 3. Choose top-k_context for prompt
    contexts = reranked[:k_context]
    
    # 4. Build prompt
    prompt = build_prompt(question, contexts)
    
    # 5. Generate answer
    answer = generate_answer(prompt)
    
    # 6. Pretty print result
    print("QUESTION:")
    print(question)
    print("\nANSWER:")
    print(textwrap.fill(answer, width=100))
    print("\n--- SOURCES ---")
    for i, c in enumerate(contexts):
        print(f"[Doc {i+1}] page={c['meta'].get('page', '?')}, score={c['score']:.3f}")
        print(textwrap.fill(c["doc"][:350], width=100))
        print("-" * 80)
    
    return {
        "answer": answer,
        "contexts": contexts
    }


In [11]:
#Test the RAG System
qa = answer_question("How do I create an index in PostgreSQL?")


QUESTION:
How do I create an index in PostgreSQL?

ANSWER:
To create an index in PostgreSQL, you must specify the following:  1. The index type: B-tree, Hash,
GiST, SP-GI-ST, GIN, BRIN, and the extension bloom.  2. The name of the index to create.  3. The
fill factor of the index.  4. The location where the index will be stored.  5. The location where
the index will reside in the tablespace.  6. The name of the column that will be used as the key for
the index.  7. The name of the column that will be used as the value for the index.  8. The name of
the column that will be used as the primary key for the index.  9. The name of the column that will
be used as the foreign key for the index.  10. The name of the column that will be used as the index
location for the index.  11. The name of the column that will be used as the index name for the
index.  12. The name of the column that will be used as the index fill factor for the index.  13.
The name of the column that will be used as the in

In [12]:
#another 
qa = answer_question("What is the purpose of VACUUM in PostgreSQL?")


QUESTION:
What is the purpose of VACUUM in PostgreSQL?

ANSWER:
The purpose of VACUUM in PostgreSQL is to reclaim space occupied by dead tuples, allowing for
efficient table scans. It also helps in maintaining a visibility map for each table, keeping track
of which pages contain only tuples that are known to be visible to all active transactions (and all
future transactions, until the page is again modified). Additionally, it can prevent loss of very
old data due to transaction ID wraparound or multixact ID wrap-around. VACUUM also supports various
options like FULL, FREEZE, VERBOSE, ANALYZE, and DISABLE_PAGE_SKIPPING, among others, depending on
the needs of the administrator. It is essential to understand the issues discussed in the previous
sections before implementing VACUUM.

--- SOURCES ---
[Doc 1] page=785, score=0.985
and MRTG, but can be run standalone too. PostgreSQL is low-maintenance compared to some other
database management systems. Nonetheless, appropriate attention to th